In [1]:
!pip install --upgrade pip
!pip install geopandas
!pip install selenium
!pip install webdriver_manager
!pip install plotly

Antes de desenvolver o grafico vou rodar um script Selenium para baixar a versão atual dos dados do site do ministerio da saude

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(ChromeDriverManager().install(),options=options)

C:\Users\pheli\AppData\Local\Temp/ipykernel_31904/674006772.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(),options=options)


In [3]:
driver.get('https://covid.saude.gov.br/')
driver.find_element(By.XPATH,'/html/body/app-root/ion-app/ion-router-outlet/app-home/ion-content/div[1]/div[2]/ion-button').click()

Agora vamos preparar os dados para o grafico

In [2]:
import pandas as pd
import os
import geopandas as gpd
import numpy as np
import plotly.express as px
from datetime import datetime
from urllib.request import urlopen
import json
PATH = 'csv/'

In [3]:
files = [f for f in os.listdir(PATH) if os.path.isfile(os.path.join(PATH, f))]
df = pd.DataFrame()
for f in files:
  df = pd.concat([df,pd.read_csv(PATH + f, encoding = "ISO-8859-1", sep = ';')])
print(df.shape)
df.dropna(subset=['municipio'],inplace=True)
print(df.shape)
df['datetime'] = pd.to_datetime(df['data'], format="%Y/%m/%d")
df.head(2)

(4805113, 17)
(4762350, 17)


,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana,datetime
4551,Norte,RO,Alta Floresta D'Oeste,11,110001.0,11005.0,ZONA DA MATA,2020-03-27,13,22945.0,0.0,0,0,0,NaN,NaN,0.0,2020-03-27
4552,Norte,RO,Alta Floresta D'Oeste,11,110001.0,11005.0,ZONA DA MATA,2020-03-28,13,22945.0,0.0,0,0,0,NaN,NaN,0.0,2020-03-28


In [4]:
with urlopen("https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson") as response:
 Brazil = json.load(response) # Javascrip object notation 

for feature in Brazil ['features']:
 feature['id'] = feature['properties']['sigla']

In [5]:
fig = px.choropleth(df.loc[df['datetime'] < datetime.strptime('2020/10/01', "%Y/%m/%d")], 
                    locations ="estado", 
                    color ="casosAcumulado", 
                    geojson = Brazil,
                    hover_name ="estado", 
                    animation_frame ="data") 

fig.update_geos(fitbounds = "locations", visible = False)
fig.show()